In [39]:
import warnings; warnings.filterwarnings("ignore")

In [40]:
import numpy as np
import pandas as pd
import xgboost; print(xgboost.__version__)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import precision_score

1.2.0


</br>

* **data processing (Merge all data)**

In [41]:
X_kmer = np.load('K-mer.npy')
X_revk = np.load('rev-k-mer.npy')
X_gapk = np.load('gapped_k_mer.npy')

In [42]:
print(X_kmer.shape)
print(X_revk.shape)
print(X_gapk.shape)

(1081, 340)
(1081, 340)
(1081, 80)


In [43]:
Y  = [1 for i in range(490)]
Y += [0 for i in range(591)]
Y = np.array(Y)
print(Y.shape)

(1081,)


In [44]:
X = np.concatenate((X_kmer,X_revk, X_gapk),axis=1)

In [45]:
X.shape

(1081, 760)

</br>

* **Shuffle**

In [46]:
from sklearn.utils import shuffle
X, Y = shuffle(X, Y, random_state=0)

print(X.shape)
print(Y.shape)

(1081, 760)
(1081,)


</br>

* **Train-Test Split**

In [47]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.30, random_state=101)

In [48]:
print(Xtrain.shape)
print(Ytrain.shape)
print(Xtest.shape)
print(Ytest.shape)

(756, 760)
(756,)
(325, 760)
(325,)


</br>

* **Scaling**

In [49]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

</br>

* **Feature Selection**

In [50]:
model = xgboost.XGBClassifier()
model.fit(Xtrain, Ytrain)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [51]:
importantFeatures = model.feature_importances_
SF = importantFeatures.argsort()[::-1][:int(len(importantFeatures)*0.4)]

In [52]:
# argsort explained
# argsort = np.array([0.9, 0.1, 0.8, 0.4, 0.6])
# print(argsort.argsort()[::-1])

In [53]:
SF.shape

(304,)

</br>

* **Model Training after Selection**

In [54]:
from sklearn.metrics import accuracy_score

model = xgboost.XGBClassifier()
model.fit(Xtrain[:, SF], Ytrain)
Yp = model.predict(Xtest[:, SF])

In [55]:
# Yp

In [56]:
Accuracy = []
Sensitivity = []
Specificity = []
Precision = []
MCC = []

CM = confusion_matrix(y_pred=Yp, y_true=Ytest)
TN, FP, FN, TP = CM.ravel()

MCC.append(matthews_corrcoef(y_true=Ytest, y_pred=Yp))
Sensitivity.append( TP / (TP + FN) )
Specificity.append( TN / (TN + FP) )
Precision.append(precision_score(y_true=Ytest, y_pred=Yp))

In [57]:
print(accuracy_score(y_true=Ytest, y_pred=Yp))
print('Sensitivity: {0:.2f}'.format(np.mean(Sensitivity)))
print('Specificity: {0:.2f}'.format(np.mean(Specificity)))
print('MCC: {0:.2f}'.format(np.mean(MCC)))
print('Precision: {0:.2f}'.format(np.mean(Precision)))

0.7446153846153846
Sensitivity: 0.60
Specificity: 0.86
MCC: 0.49
Precision: 0.79
